In [1]:
import torch
import torchvision.transforms.v2 as T

import numpy as np
import cv2, PIL
from pandas import DataFrame
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm

from skimage.feature import hog
from sklearn.model_selection import cross_val_score


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### Data Load

In [3]:
X_train = np.load('data/classification1/Xtrain_Classification1.npy')
y_train = np.load('data/classification1/ytrain_Classification1.npy')
x_test = np.load('data/classification1/Xtest_Classification1.npy')
y_train.shape

(6254,)

### Data Reshape and Split

In [4]:
data_shape = (-1,28,28,3)
X_train = X_train.reshape(data_shape)/255.
X_train = torch.FloatTensor(np.transpose(X_train, (0, 3, 1, 2))).to(device)
x_test = x_test.reshape(data_shape)/255.
x_test = torch.FloatTensor(np.transpose(x_test, (0, 3, 1, 2))).to(device)
print(x_test.shape)

torch.Size([1764, 3, 28, 28])


In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
print(X_train.type)

torch.Size([5003, 3, 28, 28]) torch.Size([1251, 3, 28, 28]) (5003,) (1251,)
<built-in method type of Tensor object at 0x0000029D6B0E1910>


In [6]:
# No SVM você treina o modelo sobre as características da imagem e 
features = []
labels = []

features_val = []
labels_val = []

features_test = []
labels_test = []

# Aplicando HOG sobre X_train
for i in range(X_train.shape[0]):
    image_tensor = X_train[i] # imagem da iteração
    image_np = image_tensor.numpy() #tensor --> numpy
    hog_features = hog(image_np, pixels_per_cell=(8, 8),channel_axis=0)#características HOG da imagem
    features.append(hog_features)
X_train_features = np.array(features) #array numpy


# Aplicando HOG sobre X_validation
for i in range(X_val.shape[0]):
    image_tensor = X_val[i] # imagem da iteração
    image_np = image_tensor.numpy() #tensor --> numpy
    hog_features_val = hog(image_np, pixels_per_cell=(8, 8),channel_axis=0)#características HOG da imagem
    features_val.append(hog_features_val)
X_val_features = np.array(features_val) #array numpy

# Aplicando HOG sobre x_test
for i in range(x_test.shape[0]):
    image_tensor = x_test[i] # imagem da iteração
    image_np = image_tensor.numpy() #tensor --> numpy
    hog_features_test = hog(image_np, pixels_per_cell=(8, 8),channel_axis=0)#características HOG da imagem
    features_test.append(hog_features_test)
X_test_features = np.array(features_test) #array numpy

In [8]:

# Crie o classificador SVM
clf = svm.SVC(gamma=0.001,C=100) # necessário trazer acaracterísticas das imagens para colocar no SVM

# Treine o modelo SVM
clf.fit(X_train_features, y_train)

y_pred = clf.predict(X_test_features)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Validation

# Acurácia
accuracy = accuracy_score(x_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%")

# Precisão
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precisão: {precision * 100:.2f}%")

# Revocação (Recall)
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Revocação (Recall): {recall * 100:.2f}%")

# F1-score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1-score: {f1 * 100:.2f}%")

# Matriz de Confusão
confusion = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(confusion)
Neste código, as métricas de acurácia, precisão, revocação (recall) e F1-score são calculadas usando as funções do scikit-learn. Além disso, a matriz de confusão é gerada para fornecer informações detalhadas sobre as previsões do modelo. Certifique-se de ajustar o valor do parâmetro average nas funções de precisão, revocação e F1-score de acordo com os requisitos do seu problema (por exemplo, 'weighted', 'micro', 'macro', etc.). Essa seleção depende da natureza dos seus dados e das classes de destino.







### Teste 1

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

(torch.Size([5003, 3, 28, 28]),
 torch.Size([1251, 3, 28, 28]),
 (5003,),
 (1251,))

### Data Augmentation

In [ ]:
y_train_df = DataFrame(y_train)
y_val_df = DataFrame(y_val)

y_val_df.value_counts()

0.0    1082
1.0     169
Name: count, dtype: int64